In [3]:
import torch

import sys
sys.path.append('..') #to add top-level to path
sys.path.append('../modules') #to add top-level to path

from modules.project_helper import TweetData

import numpy as np
import pandas as pd
import os

In [4]:
tweet_data = TweetData()

In [5]:
daily_tweets = tweet_data.clean_tweets #[pd.to_datetime(tweet_data.clean_tweets.after4_date)
                                       #<= pd.to_datetime(daily_df.index[-1])]
daily_tweets.after4_date = pd.to_datetime(daily_tweets.after4_date)

In [6]:
daily_tweets.head()

,tweets,timestamp,after4_date
timestamp,,,
2019-11-17 19:57:12-06:00,tell jennifer williams whoever that is to read...,2019-11-17 19:57:12-06:00,2019-11-18
2019-11-17 19:56:02-06:00,,2019-11-17 19:56:02-06:00,2019-11-18
2019-11-17 19:49:47-06:00,paul krugman of has been wrong about me from t...,2019-11-17 19:49:47-06:00,2019-11-18
2019-11-17 19:47:32-06:00,schiff is a corrupt politician,2019-11-17 19:47:32-06:00,2019-11-18
2019-11-17 19:30:09-06:00,blew the nasty amp obnoxious chris wallace wil...,2019-11-17 19:30:09-06:00,2019-11-18


In [7]:
model = torch.load('../models/tweet_embeddings/trump_rnn_1911.net')

In [8]:
embeddings = model.in_embed.weight.to('cpu').data.numpy()

In [9]:
embeddings.shape

(4574, 100)

In [11]:
arches = os.listdir('../models/tweet_embeddings')
arches[0:5]

['trump_rnn_1702.net',
 'trump_rnn_1703.net',
 'trump_rnn_1704.net',
 'trump_rnn_1705.net',
 'trump_rnn_1706.net']

In [12]:
embedding_list = []
for arch in arches:
    model = torch.load('../models/tweet_embeddings/{}'.format(arch))
    embedding_list.append(model.in_embed.weight.to('cpu').data.numpy())
    

In [13]:
embedding_list[0].shape

(2576, 100)

In [15]:
data = TweetData('../data/intermediate_data/trump_archive_ts/trump_archive_db_1911.csv')
words = data.words
vocab_to_int, int_to_vocab = data.vocab_to_int, data.int_to_vocab
int_words = data.int_words

In [16]:
embedding_list[-1]

array([[-0.08305283, -0.16442633, -0.07570944, ..., -0.0107758 ,
         0.19031201, -0.08310907],
       [-0.18238883, -0.13035859,  0.12322256, ..., -0.1847221 ,
         0.1905269 , -0.1955786 ],
       [-0.22082756,  0.07478237, -0.33069697, ..., -0.14277695,
         0.07682032,  0.05820784],
       ...,
       [ 0.9485307 , -0.02588199,  0.28236574, ..., -0.3737706 ,
         0.12231396,  0.58168954],
       [-0.3112953 , -0.4661302 ,  0.3573344 , ...,  0.6376106 ,
        -0.22513622, -0.07564565],
       [-0.71132535,  1.004769  , -0.1239484 , ..., -0.73008114,
         0.34569532, -0.20363104]], dtype=float32)

In [17]:
daily_tweets.after4_date

timestamp
2019-11-17 19:57:12-06:00   2019-11-18
2019-11-17 19:56:02-06:00   2019-11-18
2019-11-17 19:49:47-06:00   2019-11-18
2019-11-17 19:47:32-06:00   2019-11-18
2019-11-17 19:30:09-06:00   2019-11-18
                               ...    
2009-05-12 14:07:28-05:00   2009-05-12
2009-05-08 20:40:15-05:00   2009-05-09
2009-05-08 13:38:08-05:00   2009-05-08
2009-05-05 01:00:10-05:00   2009-05-05
2009-05-04 18:54:02-05:00   2009-05-05
Name: after4_date, Length: 28813, dtype: datetime64[ns]

In [18]:
tweet_embeddings = {}
dtws = daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]
tweet_embeddings_np = np.zeros([dtws.shape[0],100])

for i, tweet in enumerate(dtws):
    embed_sum = np.zeros(100) 
    tw = tweet.split()
    wd_count = 0
    for word in tw:
        try:
            if vocab_to_int[word] >= 25:
                embed_sum += embedding_list[-1][vocab_to_int[word]]
                wd_count += 1

        except:
            pass
    if wd_count > 0:    
        tweet_embeddings[i] = embed_sum/wd_count
        tweet_embeddings_np[i,:] = embed_sum/wd_count
    else:
        tweet_embeddings[i] = embed_sum
        tweet_embeddings_np[i,:] = embed_sum
        
            
        

In [19]:
tweet_embeddings_np.shape

(9582, 100)

In [20]:
pd.DataFrame(tweet_embeddings_np).to_csv('../data/intermediate_data/tweet_embeddings.csv')

In [21]:
tw = 1000

embedding = model.in_embed
embed_vectors = embedding.weight
magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
tweet_after = daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]
valid_vector = torch.FloatTensor(tweet_embeddings[tw])
wds = (torch.matmul(valid_vector, embed_vectors.t()) / torch.FloatTensor(magnitudes)).topk(10)
for wd in list(wds[1][0].numpy()):
    if wd> 25 :
        print(int_to_vocab[wd])
tweet_after[tw]

collusion
russia
campaign
russians
when
democrats
investigation
fisa
fbi


'democrats wrote to the ukrainian government in may urging it to continue investigations into president donald trump s alleged collusion with russia in the presidential campaign collusion later found not to exist '

In [22]:
wds

torch.return_types.topk(
values=tensor([[0.9849, 0.9564, 0.9179, 0.8600, 0.8480, 0.8418, 0.8298, 0.8290, 0.8268,
         0.8186]], grad_fn=<TopkBackward>),
indices=tensor([[ 206,  178,  162, 1150,   87,   77,  596, 1592,   12,  223]]))

In [23]:
tw = 1467

embedding = model.in_embed
embed_vectors = torch.FloatTensor(tweet_embeddings_np)
magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
tweet_after = daily_tweets.tweets[daily_tweets.after4_date >= pd.to_datetime('1-1-2017')]
valid_vector = torch.FloatTensor(tweet_embeddings[tw])
wds = (torch.matmul(valid_vector, embed_vectors.t()) / torch.FloatTensor(magnitudes))
wds[wds != wds] = 0
print(wds.topk(10))
for i in wds.topk(10).indices.numpy():
    for tweet_range in i:
        print(tweet_after[tweet_range])
        print('')


torch.return_types.topk(
values=tensor([[1.4677, 1.2926, 1.2896, 1.2749, 1.2671, 1.2668, 1.2552, 1.2538, 1.2535,
         1.2416]]),
indices=tensor([[1467, 2024, 2254,  289, 1203,  878, 1575, 1879,  290, 2399]]))
the federal reserve loves watching our manufacturers struggle with their exports to the benefit of other parts of the world has anyone looked at what almost all other countries are doing to take advantage of the good old usa our fed has been calling it wrong for too long 

with almost no inflation our country is needlessly being forced to pay a much higher interest rate than other countries only because of a very misguided federal reserve in addition quantitative tightening is continuing making it harder for our country to compete as good 

strong jobs report low inflation and other countries around the world doing anything possible to take advantage of the united states knowing that our federal reserve doesn t have a clue they raised rates too soon too often amp tightened whi

In [24]:

tweet_after[800]

'my daughter ivanka will be on tonight on at p m following the great at enjoy '

In [25]:
tweet_after[9335]

'my daughter ivanka has been treated so unfairly by she is a great person always pushing me to do the right thing terrible '

In [26]:
tweet_after[4955]

'will be interviewed tonight by on at p m eastern enjoy '

In [27]:
tweet_after[2434]

'i will be interviewed live tonight by on p m enjoy '